In [ ]:
from impact_engine_allocate.solver import solve_minimax_regret
import pandas as pd

In [ ]:
# Define investment opportunities
investment_opportunities = [
    {"id": "A", "cost": 4, "R_best": 15, "R_med": 10, "R_worst": 2, "confidence": 0.9},
    {"id": "B", "cost": 3, "R_best": 12, "R_med": 8, "R_worst": 1, "confidence": 0.6},
    {"id": "C", "cost": 3, "R_best": 9, "R_med": 6, "R_worst": 2, "confidence": 0.80},
    {"id": "D", "cost": 2, "R_best": 7, "R_med": 5, "R_worst": 3, "confidence": 0.40},
    {"id": "E", "cost": 5, "R_best": 18, "R_med": 9, "R_worst": 0, "confidence": 0.50},  # Low confidence
]

In [ ]:
investments = pd.DataFrame(investment_opportunities)
investments.set_index("id", inplace=True)
investments

In [ ]:
# Define optimization parameters
BUDGET = 10
MIN_CONFIDENCE = 0.5  # Initiatives with confidence < 0.5 will be excluded
MIN_RETURN = 4  # Minimum aggregate R_worst for selected portfolio

# Solve the optimization problem
results = solve_minimax_regret(investment_opportunities, BUDGET, MIN_CONFIDENCE, MIN_RETURN)

# Print results
print("\n--- Optimization Results ---")
print(f"Solver Status: {results['status']}")
if results["status"] == "Optimal":
    print(f"Minimized Maximum Regret: {results['objective_value']:.2f}")
    print(f"Selected Initiatives: {results['selected_initiatives']}")
    print(f"Total Cost of Selected Initiatives: {results['total_cost']:.2f}")
    print("\nEffective Returns of Selected Portfolio:")
    for s, r_val in results["total_actual_returns"].items():
        print(f"  Scenario {s}: {r_val:.2f}")

    detail = results.get("detail", {})
    v_j_star = detail.get("v_j_star", {})
    regrets = detail.get("regrets", {})

    print("\nOptimal Returns for Each Scenario (V_j_star):")
    for s, v_val in v_j_star.items():
        print(f"  Scenario {s}: {v_val:.2f}")

    print("\nCalculated Regrets for the Selected Portfolio:")
    for s, r_val in regrets.items():
        print(f"  Scenario {s}: {r_val:.2f}")

    # Verify the max regret matches the objective value
    if regrets:
        actual_max_regret = max(regrets.values())
        print(f"\nActual Max Regret: {actual_max_regret:.2f}")
        matches = abs(actual_max_regret - results["objective_value"]) < 1e-6
        print(f"Matches minimized max regret? {matches}")

else:
    print("Optimization did not find an optimal solution.")